# Note

This is basically the same as `opensea-get-collections.ipynb`, BUT this one repeats that script for each owner as defined in `init_wallet_d`. This happens lower down in the script. Until then, this is the exact same as opensea-get-`collections.ipynb`

## Retrieving First Collection

https://docs.opensea.io/reference/retrieving-collections


### Initial data gathering request

In [1]:
# Create initial search, manual for now
# Only doing this because I'm not sure if we currently save this info
username = 'ARTSNFT'
owner_url = 'opensea.io/ARTSNFT'
asset_owner_wallet_url = '0xf51e77f49c8a31174c96462dd6235cdc4df9fbc4'

In [2]:
import requests

url = "https://api.opensea.io/api/v1/collections?asset_owner={}&offset=0&limit=300".format(asset_owner_wallet_url)

headers = {"Accept": "application/json"}

response = requests.request("GET", url, headers=headers)

### Initial formatting

In [3]:
import json
import pandas as pd
import numpy as np

In [4]:
response_text = json.loads(response.text)
df = pd.json_normalize(response_text)

The dataframe looks good overall, but the 'primary_asset_contracts' column is messy. It is a column of lists with a length of 1. The only element within that list is a dictionary.

In [5]:
df.head(2)

,primary_asset_contracts,banner_image_url,chat_url,created_date,default_to_fiat,description,dev_buyer_fee_basis_points,dev_seller_fee_basis_points,discord_url,external_url,...,traits.Health.min,traits.Health.max,traits.Magic.min,traits.Magic.max,traits.Skill.min,traits.Skill.max,traits.Speed.min,traits.Speed.max,traits.Strength.min,traits.Strength.max
0,[{'address': '0x9d8826881a2beab386a7858e59c228...,https://lh3.googleusercontent.com/agQelV58HJdf...,None,2022-02-21T15:37:33.224465,False,Gen 0 Bloodshed Bears are the first Bloodshed ...,0,750,None,https://bloodshedbears.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[{'address': '0xf026fb5be56ff63e19e9da3236035e...,https://lh3.googleusercontent.com/ksoOqRn6NWvX...,None,2022-02-18T20:18:06.263313,False,"**Staking has started, true number of unique w...",0,750,https://discord.gg/shadowquest,https://www.shadowquest.games/,...,16.0,42.0,16.0,26.0,16.0,26.0,16.0,26.0,16.0,26.0


In [6]:
print("Each element in the series is", type(df['primary_asset_contracts'][0]))
print("Each of these lists has a length of", len(df['primary_asset_contracts'][0]))
print("The only element within these lists is of type", type(df['primary_asset_contracts'][0][0]))

Each element in the series is <class 'list'>
Each of these lists has a length of 1
The only element within these lists is of type <class 'dict'>


The next step is to convert this column of dictionaries (each within their own list) into a set of columns, which will be re-combined with the original dataframe. The original column, "primary_asset_contracts", will be removed from the dataframe.

The newly-created columns, which expand on the original column ("primary_asset_contracts") will all start with "pac_" before continuing their original names. The original names were the keys in the dictionaries.

In [7]:
# Create the initial dataframe from the first row
pac_df = pd.DataFrame(df.iloc[0]['primary_asset_contracts'][0], index=[0])

# Loop through the rest
for pac in df['primary_asset_contracts'][1:]:
    try:
        temp_df = pd.DataFrame(pac[0], index=[0])
# The IndexError comes from an empty list
# Solution: append an empty df with same column names
    except IndexError:
        empty_d = {}
        for col in pac_df.columns:
            empty_d[col] = np.nan
        temp_df = pd.DataFrame(empty_d, index=[0])
    pac_df = pac_df.append(temp_df)
    
# Adding "pac_" to start of each column name to track them later
pac_df.rename(columns=lambda x: "pac_"+x, inplace=True)
# All rows currently have index of 0, dropping and replacing with standard range indexes
pac_df = pac_df.reset_index(drop=True)

In [8]:
# Recombine the dataframes
df = df.join(pac_df)
# Drop the original column
df = df.drop(columns=['primary_asset_contracts'])

In [9]:
# Showing the columns just because I can't see them all with df.head()
df.columns

Index(['banner_image_url', 'chat_url', 'created_date', 'default_to_fiat',
       'description', 'dev_buyer_fee_basis_points',
       'dev_seller_fee_basis_points', 'discord_url', 'external_url',
       'featured', 'featured_image_url', 'hidden', 'safelist_request_status',
       'image_url', 'is_subject_to_whitelist', 'large_image_url',
       'medium_username', 'name', 'only_proxied_transfers',
       'opensea_buyer_fee_basis_points', 'opensea_seller_fee_basis_points',
       'payout_address', 'require_email', 'short_description', 'slug',
       'telegram_url', 'twitter_username', 'instagram_username', 'wiki_url',
       'owned_asset_count', 'stats.one_day_volume', 'stats.one_day_change',
       'stats.one_day_sales', 'stats.one_day_average_price',
       'stats.seven_day_volume', 'stats.seven_day_change',
       'stats.seven_day_sales', 'stats.seven_day_average_price',
       'stats.thirty_day_volume', 'stats.thirty_day_change',
       'stats.thirty_day_sales', 'stats.thirty_day_aver

## Adding contextual information

In [10]:
df['owner_user'] = username
df['owner_url'] = owner_url
df['asset_owner_wallet_url'] = asset_owner_wallet_url

In [11]:
# Verify the shape of the DF before adding a whole bunch more in
df.shape

(7, 87)

# Do it again for all the manual entries

Manually creating a list of wallet IDs and usernames, from a few owners of CryptoPunks, adidas Originals Into the Metaverse (top 3 in number of NFTs owned), invisible friends, and I think that's it!

In [12]:
# For rate-limiting and whatnot
from time import sleep
from random import randint

In [13]:
# FORMAT IS
# { username : (owner_url, asset_owner_wallet_url) }

init_wallet_d = {
    'PUNKVault-NFTX': ('opensea.io/PUNKVault-NFTX', '0x269616d549d7e8eaa82dfb17028d0b212d11232a'),
    'WuTangClan': ('opensea.io/WuTangClan', '0x9e67d018488ad636b538e4158e9e7577f2ecac12'),
    'Unnamed': ('opensea.io/0x7b8961f67382c51c389726667ef7dd7cf95fefd3', '0x7b8961f67382c51c389726667ef7dd7cf95fefd3'),
    '0xef764bac8a438e7e498c2e5fccf0f174c3e3f8db': ('https://opensea.io/0xef764bac8a438e7e498c2e5fccf0f174c3e3f8db', '0xef764bac8a438e7e498c2e5fccf0f174c3e3f8db'),
    'peruggia-v.eth': ('opensea.io/0xf4b4a58974524e183c275f3c6ea895bc2368e738', '0xf4b4a58974524e183c275f3c6ea895bc2368e738'),
    'Unnamed': ('opensea.io/0x1251122f1d77fa46e1e576c4fd6dd56ab17812ff', '0x1251122f1d77fa46e1e576c4fd6dd56ab17812ff'),
    'Unnamed': ('opensea.io/0x87b1702a5a31f6b63820c99f2234a3323ac728c4', '0x87b1702a5a31f6b63820c99f2234a3323ac728c4'),
    'cx': ('opensea.io/cx', '0x66c7a7348250f453a37cf86410a946f9668de386'),
    'Unnamed': ('opensea.io/0xae33cc279f43f746f4771674cf3b8200477ce017', '0xae33cc279f43f746f4771674cf3b8200477ce017'),
    'Unnamed': ('opensea.io/0x2754637ab168ff25412b74997c0e4f43c30bb323', '0x2754637ab168ff25412b74997c0e4f43c30bb323'),
    'MitchWainer': ('opensea.io/MitchWainer', '0x066d0ba423756b72cc0d78e5f15fbddb58a1851a'),
    'hipcityreg': ('opensea.io/hipcityreg', '0x06ac1f9f86520225b73efce4982c9d9505753251'),
    'Larkaiinc': ('opensea.io/Larkaiinc', '0xe7c9c928f32946a034c3bd8ef7200ad81be41afa'),
    'Unnamed': ('opensea.io/0xab62f4fb16318558b49a2b4dad45ed237438b79d', '0xab62f4fb16318558b49a2b4dad45ed237438b79d'),
    'Lucaesti': ('opensea.io/Lucaesti', '0x534c8bc9781a8072b524b853147a69bc6bf2b552'),
    'q': ('opensea.io/q', '0xa52899a1a8195c3eef30e0b08658705250e154ae'),
    'ricky': ('opensea.io/ricky', '0x7d4823262bd2c6e4fa78872f2587dda2a65828ed'),
    'Jake Paul': ('opensea.io/Jake-Paul', '0xd81e1713c99595ee29498e521b18491af9c60415'),
    'Cozomo de Medici': ('opensea.io/Cozomo_de_Medici', '0xce90a7949bb78892f159f428d0dc23a8e3584d75'),
    'Kushari': ('opensea.io/Kushari', '0x51a8ed1ac9f120bf8b69edc654dd6809ab5db41c'),
    'LilMayo': ('opensea.io/LilMayo', '0xa582047f7e50acbc8667523e7f62c200709beaed'),
    'Jordan Beflort': ('opensea.io/0xdbf2445e5049c04cda797dae60ac885e7d79df9d', '0xdbf2445e5049c04cda797dae60ac885e7d79df9d'),
    'imnotscottysire': ('opensea.io/imnotscottysire', '0xd0a1454963fb17f427fe744a084facd0ed60a774'),
    'BenPhillipsUK': ('opensea.io/BenPhillipsUK', '0x53295d7932767839df07eb175ef03a24dcf3c278'),
    'LordTJVault': ('opensea.io/LordTJVault', '0x5c5954af429ea585badec7fb13790f0d7d1569e8'),
    'MikeShinoda': ('opensea.io/MikeShinoda', '0xb55eb9bd32d6ab75d7555192e7a3a7ca0bcd5738'),
    'Snipe_Snipe': ('opensea.io/Snipe_Snipe', '0xe033b12daf37e64d6e664ac5b8eb839ce5b749db')
}

In [24]:
len(init_wallet_d.keys())

22

In [14]:
for item in init_wallet_d.items():
#     variable init
    username = item[0]
    owner_url = item[1][0]
    asset_owner_wallet_url = item[1][1]
    
#     creating request url and headers
    url = "https://api.opensea.io/api/v1/collections?asset_owner={}&offset=0&limit=300".format(asset_owner_wallet_url)
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers)
    
#     initial formatting

    response_text = json.loads(response.text)
    temp_df = pd.json_normalize(response_text)
    
    
###     Clean that annoying primary_asset_contracts stuff
    # Create the initial dataframe from the first row
    try:
        pac_df = pd.DataFrame(temp_df.iloc[0]['primary_asset_contracts'][0], index=[0])

        # Loop through the rest
        for pac in temp_df['primary_asset_contracts'][1:]:
            try:
                temp_df = pd.DataFrame(pac[0], index=[0])
        # The IndexError comes from an empty list
        # Solution: append an empty df with same column names
            except IndexError:
                empty_d = {}
                for col in pac_df.columns:
                    empty_d[col] = np.nan
                temp_df = pd.DataFrame(empty_d, index=[0])
            pac_df = pac_df.append(temp_df)

        # Adding "pac_" to start of each column name to track them later
        pac_df.rename(columns=lambda x: "pac_"+x, inplace=True)
        # All rows currently have index of 0, dropping and replacing with standard range indexes
        pac_df = pac_df.reset_index(drop=True)

        # Recombine the dataframes
        temp_df = temp_df.join(pac_df)
        # Drop the original column
        if 'primary_asset_contracts' in temp_df.columns:
            temp_df = temp_df.drop(columns=['primary_asset_contracts'])
    
    except IndexError:
        pass
    
# Adding contextual information
    temp_df['owner_user'] = username
    temp_df['owner_url'] = owner_url
    temp_df['asset_owner_wallet_url'] = asset_owner_wallet_url
    
# Formatting and clarifying column names
    temp_df = temp_df.rename(columns={'name':'collection_name'})
    
    
# Add temp_df to the original, large main DF, called df
    df = df.append(temp_df)
    
# Let's sleep for a random amount of time
    sleep(randint(3, 10))

## Check that it all looks good

In [15]:
print(df.shape)

(775, 2331)


There are a few columns with all empty values. I'm going to remove them

In [16]:
for col in df.columns:
    if col not in df.dropna(axis=1, how='all').columns:
        print(col)

chat_url
pac_opensea_version
opensea_version


The code also grabs a whole bunch of "traits", which I believe are properties for a specific NFT. These may be useful in predicting price, so I'm going to save them in another dataframe, `traits_df` and remove them from this main one

In [17]:
# Create the traits_df
traits_df = df[[col for col in df.columns if 'traits' in col]].copy()

# Remove those columns from traits_df
df = df[df.columns.drop(list(df.filter(regex='traits')))]

# Some columns are totally empty. 
df = df.dropna(axis=1, how='all')

# Currently, empty cols are 

print("the traits_df has a shape of", traits_df.shape)
print("the main df has a shape of", df.shape)

the traits_df has a shape of (775, 2240)
the main df has a shape of (775, 88)


In [18]:
print(df.shape)

(775, 88)


## Export

In [19]:
# Exporting to csv
df.to_csv('data/opensea_get_many_collections.csv', index=False)

# Exporting the traits df to csv because why not
traits_df.to_csv('data/opensea_traits.csv', index=False)

## Observing what data we have now

In [20]:
df.head()

,banner_image_url,created_date,default_to_fiat,description,dev_buyer_fee_basis_points,dev_seller_fee_basis_points,discord_url,external_url,featured,featured_image_url,...,address,asset_contract_type,nft_version,owner,schema_name,symbol,total_supply,external_link,buyer_fee_basis_points,seller_fee_basis_points
0,https://lh3.googleusercontent.com/agQelV58HJdf...,2022-02-21T15:37:33.224465,False,Gen 0 Bloodshed Bears are the first Bloodshed ...,0,750,None,https://bloodshedbears.com/,False,https://lh3.googleusercontent.com/-mtSJ6aFZGP_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://lh3.googleusercontent.com/ksoOqRn6NWvX...,2022-02-18T20:18:06.263313,False,"**Staking has started, true number of unique w...",0,750,https://discord.gg/shadowquest,https://www.shadowquest.games/,False,https://lh3.googleusercontent.com/0V1m53ay3HEQ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://lh3.googleusercontent.com/SwWOvZt1NQeu...,2022-02-18T18:15:27.899604,False,"8k invisible mfers vibin on metaverse, #mfers...",0,250,https://discord.gg/AW9HhFGh,None,False,https://lh3.googleusercontent.com/dlczqb6Fk6Zj...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://lh3.googleusercontent.com/Mr4UVY-aWCW-...,2022-02-15T19:15:33.655423,False,Welcome to the cute and pudgy world of CatBlox...,0,750,None,https://www.catblox.xyz,False,https://lh3.googleusercontent.com/lYfTzWEiXVN7...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://lh3.googleusercontent.com/bDun3SlP1OI9...,2022-02-10T21:37:55.411948,False,NASA is a Play-to-Earn metaverse game\nhttps:/...,0,750,https://discord.gg/notoriousaliens,http://notoriousaliens.com/,False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df.columns

Index(['banner_image_url', 'created_date', 'default_to_fiat', 'description',
       'dev_buyer_fee_basis_points', 'dev_seller_fee_basis_points',
       'discord_url', 'external_url', 'featured', 'featured_image_url',
       'hidden', 'safelist_request_status', 'image_url',
       'is_subject_to_whitelist', 'large_image_url', 'medium_username', 'name',
       'only_proxied_transfers', 'opensea_buyer_fee_basis_points',
       'opensea_seller_fee_basis_points', 'payout_address', 'require_email',
       'short_description', 'slug', 'telegram_url', 'twitter_username',
       'instagram_username', 'wiki_url', 'owned_asset_count',
       'stats.one_day_volume', 'stats.one_day_change', 'stats.one_day_sales',
       'stats.one_day_average_price', 'stats.seven_day_volume',
       'stats.seven_day_change', 'stats.seven_day_sales',
       'stats.seven_day_average_price', 'stats.thirty_day_volume',
       'stats.thirty_day_change', 'stats.thirty_day_sales',
       'stats.thirty_day_average_price', 

In [22]:
len(df['collection_name'].unique())

695

In [23]:
df.shape

(775, 88)